# 决策树

## 概述
决策树（Decision Tree）算法是一种基本的分类与回归方法，这章节只讨论用于分类的决策树。

决策树模型呈树形结构，在分类问题中，表示基于特征对实例进行分类的过程。它可以认为是 if-then 规则的集合，也可以认为是定义在特征空间与类空间上的条件概率分布。

决策树学习通常包括 3 个步骤: 特征选择、决策树的生成和决策树的修剪。

## 定义
决策树由结点（node）和有向边（directed edge）组成。结点有两种类型: 内部结点（internal node）和叶结点（leaf node）。内部结点表示一个特征或属性(features)，叶结点表示一个类(labels)。

用决策树对需要测试的实例进行分类: 
-   从根节点开始，对实例的某一特征进行测试，根据测试结果，将实例分配到其子结点；
-   这时，每一个子结点对应着该特征的一个取值。
-   递归地对实例进行测试并分配，直至达到叶结点。最后将实例分配到叶结点的类。

## 原理
### 信息熵 & 信息增益
-   信息论（information theory）中的熵（香农熵）: 是一种信息的度量方式，表示信息的混乱程度，也就是说: 信息越有序，信息熵越低；信息越无序，信息熵越高
-   信息增益（information gain）: 在划分数据集前后信息发生的变化称为信息增益。

## 算法特点
-   优点: 计算复杂度不高，输出结果易于理解，数据有缺失也能跑，可以处理不相关特征。
-   缺点: 容易过拟合。
-   适用数据类型: 数值型和标称型。

# 实战1
## 概述
根据以下 2 个特征，将动物分成两类: 鱼类和非鱼类。
-   不浮出水面是否可以生存
-   是否有脚蹼

In [1]:
import operator
from math import log
from collections import Counter

In [2]:
def createDataSet():
    dataSet = [[1, 1, 'yes'],
            [1, 1, 'yes'],
            [1, 0, 'no'],
            [0, 1, 'no'],
            [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels

# 计算给定数据集的香农熵
# H = -Σp(x)log2p(x), x属于类别集合
def calShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        # 为所有可能的分类创建字典
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1

    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key]) / numEntries
        shannonEnt -= prob * log(prob, 2)

    return shannonEnt

# 按照给定特征划分数据集
def splitDataSet(dataSet, index, value):
    retDataSet = []
    for featVec in dataSet:
        # 找到指定特征列为value的数据，将其余特征列组成新的数据集
        if featVec[index] == value:
            reducedFeatVec = featVec[:index]
            '''
            extend和append的区别:
            music_media.append(object) 向列表中添加一个对象object
            music_media.extend(sequence) 把一个序列seq的内容添加到列表中 (跟 += 在list运用类似， music_media += sequence)
            1、使用append的时候，是将object看作一个对象，整体打包添加到music_media对象中。
            2、使用extend的时候，是将sequence看作一个序列，将这个序列和music_media序列合并，并放在其后面。
            music_media = []
            music_media.extend([1,2,3])
            print music_media
            #结果: 
            #[1, 2, 3]
            
            music_media.append([4,5,6])
            print music_media
            #结果: 
            #[1, 2, 3, [4, 5, 6]]
            
            music_media.extend([7,8,9])
            print music_media
            #结果: 
            #[1, 2, 3, [4, 5, 6], 7, 8, 9]
            '''
            reducedFeatVec.extend(featVec[index+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

# 选择最好的数据集划分方式
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1
    # 数据集的原始信息熵
    baseEntropy = calShannonEnt(dataSet)
    # 最优的信息增益值, 和最优的Feature编号
    baseInfoGain, bestFeature = 0.0, -1

    for i in range(numFeatures):
        # 获取第i个特征的所有可能取值
        featList = [example[i] for example in dataSet]
        # 去重
        uniqueVals = set(featList)

        newEntropy = 0.0
        # 对每个特征值划分数据集，计算信息熵
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet) / float(len(dataSet))
            newEntropy += prob * calShannonEnt(subDataSet)

        infoGain = baseEntropy - newEntropy
        if infoGain > baseInfoGain:
            baseInfoGain = infoGain
            bestFeature = i
    return bestFeature

# 辅助函数，返回出现次数最多的类别
def majorityCnt(classList):

# 创建树
def createTree(dataSet, labels):
    classList = [example[-1] for example in dataSet]
    # 停止条件1：所有类标签完全相同
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    
    # 停止条件2：使用完了所有特征, 返回出现次数最多的类标签
    if len(dataSet) == 1:
        return majorityCnt(classList)

        
